<a href="https://colab.research.google.com/github/Amirnaderiy/CNN_ViT/blob/main/ViT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install opencv-python
!pip install efficientnet
!pip install vit_keras
!pip install tensorflow_addons
!pip install transformers  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
 
from vit_keras import vit, utils
import tensorflow as tf
import os , shutil
import cv2
from keras.optimizers import Adam
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.applications.efficientnet import preprocess_input
from sklearn.model_selection import train_test_split
from transformers import ViTFeatureExtractor, ViTForImageClassification
import matplotlib.pyplot as plt
from torchvision import transforms
import numpy as np
import math
import torch
import torch.nn as nn
import torch.optim as optim
from google.colab import files
from google.colab import drive


In [3]:
drive.mount('/content/drive')


Mounted at /content/drive


In [26]:
original_dataset_dir = '/content/drive/MyDrive/B-Mode Ultrasound/'


# Define the four groups based on the filename
group0 = [filename for filename in os.listdir(original_dataset_dir) if filename.startswith('Group0')]
group1 = [filename for filename in os.listdir(original_dataset_dir) if filename.startswith('Group1')]
group2 = [filename for filename in os.listdir(original_dataset_dir) if filename.startswith('Group2')]
group3 = [filename for filename in os.listdir(original_dataset_dir) if filename.startswith('Group3')]

# Define a function to load the images and resize them to a specific size
def load_and_resize_image(filename, target_size):
    img = cv2.imread(os.path.join(original_dataset_dir, filename))
    img = cv2.resize(img, target_size)
    return img

# Define the target size for the images
target_size = (224, 224)

# Load the images for each group and resize them
group0_images = [load_and_resize_image(filename, target_size) for filename in group0]
group1_images = [load_and_resize_image(filename, target_size) for filename in group1]
group2_images = [load_and_resize_image(filename, target_size) for filename in group2]
group3_images = [load_and_resize_image(filename, target_size) for filename in group3]

# Define the labels for each group
group0_labels = [0] * len(group0_images)
group1_labels = [1] * len(group1_images)
group2_labels = [2] * len(group2_images)
group3_labels = [3] * len(group3_images)

images = group0_images + group1_images + group2_images + group3_images
labels = group0_labels + group1_labels + group2_labels + group3_labels

# Split the data into train and test sets
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.30, random_state=42)

# Split the train set into train and validation sets
test_images, val_images, test_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.50, random_state=42)



image_size = 224
num_classes=4
# Load the pre-trained VIT model with imagenet weights
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-large-patch16-224')

# Load the ViT model
model = ViTForImageClassification.from_pretrained('google/vit-large-patch16-224')

# Replace the classification head with your own
model.classifier = nn.Sequential(
    nn.Linear(in_features=2048, out_features=256),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(in_features=256, out_features=num_classes))



# Define the batch size for training and validation
batch_size = 32

# Define the number of classes
num_classes = 4

# Define the data generators for training, validation, and testing
train_transform = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.5, contrast=0.5),
    transforms.RandomAffine(degrees=0, translate=(0.2, 0.2), scale=(0.8, 1.2), shear=0.2),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Define the data augmentations for the test and validation sets
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])


val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])


def train_generator(train_images, train_labels, batch_size):
    num_batches = len(train_images) // batch_size
    for i in range(num_batches):
        batch_images = train_images[i*batch_size : (i+1)*batch_size]
        batch_labels = train_labels[i*batch_size : (i+1)*batch_size]
        yield torch.from_numpy(np.array(batch_images)).to(device), torch.from_numpy(np.array(batch_labels)).to(device)

def validation_generator(val_images, val_labels, batch_size):
    num_batches = len(val_images) // batch_size
    for i in range(num_batches):
        batch_images = val_images[i*batch_size : (i+1)*batch_size]
        batch_labels = val_labels[i*batch_size : (i+1)*batch_size]
        yield torch.from_numpy(np.array(batch_images)).to(device), torch.from_numpy(np.array(batch_labels)).to(device)

def test_generator(test_images, test_labels, batch_size):
    num_batches = len(test_images) // batch_size
    for i in range(num_batches):
        batch_images = test_images[i*batch_size : (i+1)*batch_size]
        batch_labels = test_labels[i*batch_size : (i+1)*batch_size]
        yield torch.from_numpy(np.array(batch_images)).to(device), torch.from_numpy(np.array(batch_labels)).to(device)

    
# Set up optimizer and loss function
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs = 250
print_every = 10
   
    
# Training loop
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_generator(train_images, train_labels, batch_size)):
        # Zero out gradients
        optimizer.zero_grad()
        
        # Forward pass
        output = model(data)
        
        # Compute loss
        loss = criterion(output, targets)
        
        # Backward pass
        loss.backward()
        
        # Update parameters
        optimizer.step()
        
        # Print loss
        if batch_idx % print_every == 0:
            print('Epoch {} Batch {} Loss: {:.4f}'.format(epoch+1, batch_idx, loss.item()))


# Train the model
history = model.fit(train_generator, 
                    steps_per_epoch=len(train_generator), 
                    epochs=num_epochs, 
                    validation_data=validation_generator,
                    callbacks=[lr_scheduler])



test_loss, test_acc = model.evaluate(test_generator, steps=len(test_generator))



/usr/local/lib/python3.9/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


ValueError: ignored

In [ ]:
history_dict=history.history
accuracy_values=history_dict['accuracy']
val_accuracy_values=history_dict['val_accuracy']
test_accuracy_values = [test_acc] * len(accuracy_values)
epochs=range(1,len(accuracy_values)+1)
plt.plot (epochs,accuracy_values,'bo',label='Training Accuracy')
plt.plot (epochs,val_accuracy_values,'r',label='Validation Accuracy')
plt.plot (epochs,test_accuracy_values,'g',label='Test Accuracy')
plt.title('Training, Validation and Test Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

accuracy_values=history_dict['loss']
val_accuracy_values=history_dict['val_loss']
test_loss_values = [test_loss] * len(accuracy_values)
epochs=range(1,len(accuracy_values)+1)
plt.plot (epochs,accuracy_values,'b*',label='Training Loss')
plt.plot (epochs,val_accuracy_values,'r',label='Validation Loss')
plt.plot (epochs,test_loss_values,'g',label='Test Loss')
plt.title('Training, Validation and Test Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()